In [2]:
import clsDatabase as db
import pandas as pd
import datetime

In [3]:
query = 'SELECT * FROM TradingTest'

table = db.SelectQuery(query)

db.ConnectionClose()

In [13]:
data = pd.DataFrame(table)

for i, rows in data.iterrows():
    rows['candle_date_time_utc']

2021-12-12 12:45:00
2021-12-12 12:30:00
2021-12-12 12:15:00
2021-12-12 12:00:00
2021-12-12 11:45:00
2021-12-12 11:30:00
2021-12-12 11:15:00
2021-12-12 11:00:00
2021-12-12 10:45:00
2021-12-12 10:30:00
2021-12-12 10:15:00
2021-12-12 10:00:00
2021-12-12 09:45:00
2021-12-12 09:30:00
2021-12-12 09:15:00
2021-12-12 09:00:00
2021-12-12 08:45:00
2021-12-12 08:30:00
2021-12-12 08:15:00
2021-12-12 08:00:00
2021-12-12 07:45:00
2021-12-12 07:30:00
2021-12-12 07:15:00
2021-12-12 07:00:00
2021-12-12 06:45:00
2021-12-12 06:30:00
2021-12-12 06:15:00
2021-12-12 06:00:00
2021-12-12 05:45:00
2021-12-12 05:30:00
2021-12-12 05:15:00
2021-12-12 05:00:00
2021-12-12 04:45:00
2021-12-12 04:30:00
2021-12-12 04:15:00
2021-12-12 04:00:00
2021-12-12 03:45:00
2021-12-12 03:30:00
2021-12-12 03:15:00
2021-12-12 03:00:00
2021-12-12 02:45:00
2021-12-12 02:30:00
2021-12-12 02:15:00
2021-12-12 02:00:00
2021-12-12 01:45:00
2021-12-12 01:30:00
2021-12-12 01:15:00
2021-12-12 01:00:00
2021-12-12 00:45:00
2021-12-12 00:30:00
